In [8]:
import sqlite3
from sqlite3 import Error
import pandas as pd

In [2]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn


In [5]:
#https://www.kaggle.com/hugomathien/soccer
database = r"database.sqlite"
conn= create_connection(database)

In [12]:
def get_table__as_df(table_name,cnx):
    return pd.read_sql_query("SELECT * FROM " + table_name, cnx)


In [7]:
cur = conn.cursor()
cur.execute("SELECT name FROM sqlite_master WHERE type ='table' AND name NOT LIKE 'sqlite_%'")

rows = cur.fetchall()

for row in rows:
    print(row)

('Player_Attributes',)
('Player',)
('Match',)
('League',)
('Country',)
('Team',)
('Team_Attributes',)


In [14]:
df_country = get_table__as_df('Country', conn)
df_country

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy
5,13274,Netherlands
6,15722,Poland
7,17642,Portugal
8,19694,Scotland
9,21518,Spain


In [17]:
df_p = get_table__as_df('Player_Attributes', conn)
df_team = get_table__as_df('Team', conn)

In [18]:
df = pd.read_sql_query("SELECT * FROM Match", conn)
df

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25977,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df['Home_Team'] = df['home_team_api_id'].apply(lambda x: df_team[df_team['team_api_id'] == x ]['team_short_name'].values[0])
df['Away_Team'] = df['away_team_api_id'].apply(lambda x: df_team[df_team['team_api_id'] == x ]['team_short_name'].values[0])

In [25]:
df['Away_Team'].head()

0        GEN
1        ZUL
2        CEB
3        GEN
4        DEN
        ... 
25974    GAL
25975    VAD
25976    GRA
25977    LUG
25978     YB
Name: Home_Team, Length: 25979, dtype: object

In [42]:
columns_to_remove = [c for c in list(df.columns) if 'player' in c]
df = df.drop(columns_to_remove,axis =1)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,Home_Team,Away_Team
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20,GEN,BAC
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60,ZUL,LOK
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75,CEB,AND
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50,GEN,MON
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67,DEN,STL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GAL,THU
25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VAD,LUZ
25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,SIO
25977,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LUG,ZUR


In [57]:
def get_winner(row):
    if row['home_team_goal'] > row['away_team_goal']:
        return 'H'
    elif row['home_team_goal'] < row['away_team_goal']:
        return 'A'
    else:
        return 'D'


In [58]:
df['FTR'] = df.apply(get_winner,axis=1)

In [63]:
columns = ['home_team_api_id','away_team_api_id','Away_Team','Home_Team','date','home_team_goal','away_team_goal']

In [77]:
df_home_wins = df[~df['FTR'].str.contains('A')][columns].copy()
df_home_wins.drop(['away_team_goal','away_team_api_id','Away_Team'],axis=1,inplace=True)
df_home_wins.rename(columns={'home_team_api_id': 'team_id','Home_Team':'Team','home_team_goal':'goals'},inplace=True)
df_away_wins = df[~df['FTR'].str.contains('H')][columns].copy()
df_away_wins.drop(['home_team_goal','home_team_api_id','Home_Team'],axis=1,inplace=True)
df_away_wins.rename(columns={'away_team_api_id': 'team_id','Away_Team':'Team','away_team_goal':'goals'},inplace=True)

In [80]:
df_final = pd.concat([df_home_wins,df_away_wins])

In [81]:
df_final

,team_id,Team,date,goals
0,9987,GEN,2008-08-17 00:00:00,1
1,10000,ZUL,2008-08-16 00:00:00,0
3,9991,GEN,2008-08-17 00:00:00,5
5,8203,MEC,2008-09-24 00:00:00,1
6,9999,ROS,2008-08-16 00:00:00,2
...,...,...,...,...
25966,10179,SIO,2015-08-30 00:00:00,2
25971,9956,GRA,2015-09-13 00:00:00,3
25972,10191,THU,2015-09-13 00:00:00,3
25975,10199,LUZ,2015-09-23 00:00:00,2


In [85]:
df_final.groupby(['team_id','Team']).agg({'goals': ['sum','mean']})

goals          
               sum      mean
team_id Team                
1601    CHO    237  1.590604
1773    O-H     96  1.920000
1957    BIA    242  1.602649
2033    OLH    109  1.252874
2182    POZ    350  1.902174
...            ...       ...
158085  ARO     81  1.421053
177361  TBN     25  1.470588
188163  TON     23  1.642857
208931  CAP     24  1.200000
274581  MOP     60  2.142857

[299 rows x 2 columns]

In [90]:
df[columns]

,home_team_api_id,away_team_api_id,Away_Team,Home_Team,date,home_team_goal,away_team_goal
0,9987,9993,BAC,GEN,2008-08-17 00:00:00,1,1
1,10000,9994,LOK,ZUL,2008-08-16 00:00:00,0,0
2,9984,8635,AND,CEB,2008-08-16 00:00:00,0,3
3,9991,9998,MON,GEN,2008-08-17 00:00:00,5,0
4,7947,9985,STL,DEN,2008-08-16 00:00:00,1,3
...,...,...,...,...,...,...,...
25974,10190,10191,THU,GAL,2015-09-22 00:00:00,1,0
25975,9824,10199,LUZ,VAD,2015-09-23 00:00:00,1,2
25976,9956,10179,SIO,GRA,2015-09-23 00:00:00,2,0
25977,7896,10243,ZUR,LUG,2015-09-22 00:00:00,0,0


In [91]:
 df_p = get_table__as_df('Player_Attributes', conn)

In [92]:
df_p

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183973,183974,102359,39902,2009-08-30 00:00:00,83.0,85.0,right,medium,low,84.0,...,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183974,183975,102359,39902,2009-02-22 00:00:00,78.0,80.0,right,medium,low,74.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183975,183976,102359,39902,2008-08-30 00:00:00,77.0,80.0,right,medium,low,74.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183976,183977,102359,39902,2007-08-30 00:00:00,78.0,81.0,right,medium,low,74.0,...,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0
